# Wheel Test 2018-09-27
Testing new magic wheels on Jif.

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
date = '2018-09-27'
activity_meta = pd.read_pickle("data/meta-%s.pkl.gz" % date)
activity_data = pd.read_pickle("data/data-%s.pkl.gz" % date)

## TPI Grouped by Direction
Select and pivot summary data for an activity.

In [ ]:
pivot = activity_data.pivot(index='id', columns='measure', values='value')
activity = pd.merge(activity_meta, pivot, on='id')
activity['tpi'] = activity['actual_ticks'] / activity['actual_distance']
activity[['direction','actual_distance', 'tpi']].groupby('direction').describe().round(1)

In [ ]:
activity.boxplot(column='tpi', by=['name', 'direction'], rot=15, figsize=(11, 8))

## TPI Estimated Error by Direction
A resonable choice for nominal TPI is either *forward* TPI or TPI *grand mean*. We'll assume the TPI nominal value is the grand mean of all TPI measurements.

\begin{equation*}
error_{TPI} =  \frac{TPI_{experimental} - TPI_{nominal}}{TPI_{nominal}} \times 100\%
\end{equation*}

A positive error is more ticks per inch (conversely fewer inches per tick), therefore the robot travels less distance with postive error. The `100 in.` column below is how far we'd expect the robot to travel if we converted 100 in to ticks using the forward (0.0) TPI as our benchmark.

In [ ]:
tpi_grand_mean = activity['tpi'].mean()
tpi_error = activity[['direction', 'tpi']].groupby('direction').mean()
tpi_error['error'] = (tpi_error['tpi'] - tpi_grand_mean) / tpi_grand_mean
tpi_error['error (%)'] = pd.Series(["{0:.2f}%".format(val * 100) for val in tpi_error['error']], index = tpi_error.index)

ticks_100_in = tpi_error.loc['0.0','tpi'] * 100 # ticks to travel 100in using 0.0 TPI
tpi_error['100 in.'] = ticks_100_in / tpi_error['tpi']
tpi_error = tpi_error.round({'tpi':1, 'error': 3, '100 in.': 2})
tpi_error